## TYTAN tutorial おすすめ1（最大カット問題）

最終更新：2023年6月25日 by ビネクラ安田

取り組み方

*   Google Colabで取り組む場合：ファイル＞ドライブにコピーを保存　<font color="red">※このファイルを直接編集しても保存されません</font>
*   Jupyter Notebookに移す場合：ファイル＞ダウンロード＞.ipynbをダウンロード

参考リンク１

*   [TYTANチュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)
*   [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

出展

*   [量子アニーリングにおける最大カット問題（Max cut問題）を易しく解説](https://vigne-cla.com/21-13/)



### 問題
QUBOでこの問題を解く。「できるだけ多くの線を切る」と説明されることが多いが、ちょっと分かりにくい。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-13_1-300x189.png" width = 30%>
</div>

最適解の一つはこちら。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-13_2.png" width = 30%>
</div>

でも、もう一箇所切れるんじゃね？とも思える。でもそれは不正解。なぜだろう？と混乱する。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-13_3.png" width = 30%>
</div>

### 問題の言い換え
５人の幼稚園児を２台のバスに乗せます。友達関係をできるだけ壊すように振り分けるには、どのようにグループ分けしたら良いでしょうか？

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-13_4l.png" width = 35%>
</div>

### このQUBO設定を使おう

<font color="red">「n個の量子ビットからm個を1にする」</font>

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件

５人の幼稚園児を５個の量子ビットに対応させる。

次に、ある友達関係に着目したとき、彼らを違うバスに乗せたいということは、彼らの量子ビットを0, 1逆にしたいということで、さらに言い換えると**「2個の量子ビットから1個を1にしたい」**と同じ。これをすべての友達関係で設定する。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-13_5rl-768x327.png" width = 60%>
</div>

（補足）今回は友達関係の強さに差がなく、どの線も同じ重み。結果的に「できるだけ多くの本数を切る」となる。より一般化された最大カット問題は友情に差があるため、それに応じた係数をかける。「こっちを2本切ってあっちを1本残そう」といったことも起こり得る。


## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [10]:
from tytan import *

#量子ビットを用意する
q0 = symbols('q0')
q1 = symbols('q1')
q2 = symbols('q2')
q3 = symbols('q3')
q4 = symbols('q4')

#友達関係において、違うバスに乗せたい（＝2個の量子ビットを0,1逆にしたい）（＝2個の量子ビットから1個を1にしたい）
H = 0
H += (q0 + q1 - 1)**2
H += (q0 + q2 - 1)**2
H += (q1 + q3 - 1)**2
H += (q2 + q3 - 1)**2
H += (q2 + q4 - 1)**2
H += (q3 + q4 - 1)**2


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#結果
for r in result:
    print(r)

[{'q0': 0, 'q1': 1, 'q2': 1, 'q3': 0, 'q4': 0}, -5.0, 26]
[{'q0': 0, 'q1': 1, 'q2': 1, 'q3': 0, 'q4': 1}, -5.0, 23]
[{'q0': 1, 'q1': 0, 'q2': 0, 'q3': 1, 'q4': 0}, -5.0, 23]
[{'q0': 1, 'q1': 0, 'q2': 0, 'q3': 1, 'q4': 1}, -5.0, 28]


4種類の最適解が得られるが実質2パターン。一方は冒頭の模範解答と一致していて、もう一方は別解。

エネルギーが-5になったことについて。まずオフセットで-6されていて、各条件式は叶うと+0、叶わないと+1される。つまり、5式が叶って1式が叶わなかった（5本切れて1本残った）ことを意味する。